In [6]:
import os
import cv2
import numpy as np
from insightface.app import FaceAnalysis

def load_model():
    app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    app.prepare(ctx_id=0, det_size=(640, 640))
    return app

def apply_gaussian_blur(img, kernel_size=(5, 5)):
    return cv2.GaussianBlur(img, kernel_size, 0)

def add_gaussian_noise(img):
    row, col, ch = img.shape
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy_img = img + gauss * 255
    return np.clip(noisy_img, 0, 255).astype(np.uint8)

def detect_and_process_faces(model, img_path, output_folder, target_size=(128, 128)):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Unable to load image at {img_path}. Skipping...")
        return
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = model.get(img_rgb)
    degraded_img = add_gaussian_noise(apply_gaussian_blur(img))

    if not faces:
        print(f"No faces detected in {img_path}. Skipping...")
        return

    for idx, face in enumerate(faces):
        bbox = face.bbox.astype(int)
        x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]

        # Ensure the bounding box is valid
        if x1 >= x2 or y1 >= y2:
            print(f"Invalid bounding box for face {idx} in image {img_path}. Skipping...")
            continue

        cropped_face = degraded_img[y1:y2, x1:x2]

        # Check if the cropped image is empty
        if cropped_face.size == 0:
            print(f"Cropped face {idx} is empty in image {img_path}. Skipping...")
            continue

        # Resize the cropped face to the target size
        resized_face = cv2.resize(cropped_face, target_size)

        face_path = os.path.join(output_folder, f"face_{idx}_{os.path.basename(img_path)}")
        cv2.imwrite(face_path, resized_face)
        print(f"Processed and saved: {face_path}")

def main(input_folder, output_folder):
    model = load_model()
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for image_file in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_file)
        detect_and_process_faces(model, image_path, output_folder)

if __name__ == "__main__":
    input_folder = r'F:\super_resolution_project\hr_imgs'
    output_folder = r'F:\super_resolution_project\lr_imgs'
    main(input_folder, output_folder)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\STARIZ.PK/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\STARIZ.PK/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\STARIZ.PK/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\STARIZ.PK/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\STARIZ.PK/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None